# **Part A: Installing Packages and Basic Visualization of ECG**

## **A1: Installing Packages**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
from os.path import join as osj

# Base folder in your Drive where you put the unzipped MIT-BIH files
PROJECT_PATH = "/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0/"     # <-- change folder name if yours is different
DATA_ROOT    = PROJECT_PATH    # the folder that contains 100.dat, 100.hea, etc.
os.makedirs(DATA_ROOT, exist_ok=True)

# Build the list of available record IDs (e.g., '100', '101', ...)
RECORDS = sorted({os.path.splitext(f)[0] for f in os.listdir(DATA_ROOT) if f.endswith(".dat")})

print("PROJECT_PATH:", PROJECT_PATH)
print("DATA_ROOT   :", DATA_ROOT)
print("Total records found:", len(RECORDS))
print("Sample:", RECORDS[:10])

PROJECT_PATH: /content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0/
DATA_ROOT   : /content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0/
Total records found: 0
Sample: []


In [ ]:
project_path = DATA_ROOT
patient_ids = RECORDS

In [ ]:
# wfdb is not normally installed in Colab
!pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 148.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [ ]:
# Importing packages
import os
import datetime
import wfdb
import pywt
import seaborn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from os.path import join as osj
import pandas as pd

## **A2: Basic Visualization of ECG**
Basic code for loading (reading), plotting and playing with ECG signals

### **a. Getting Recordings' IDs**
The ECG recordings are named after Patients' IDs (from 100 to 234), sorted but not consecutive. Total 48 recordings.

In [ ]:
patient_ids = RECORDS
patient_ids

[]

### **b. 1 Patient ECG loading and plotting**
Extracting 2 leads ECG signals of a patient (for example: 100), and saving in two lists.

In [ ]:
#Extracting just 1 patient ECG signal and info
lead0 = {}  # without this it shows lead0[100] is not defined
lead1 = {}
patient_id = "100"
signals, info = wfdb.io.rdsamp(osj(DATA_ROOT, str(100)))
lead0[100] = signals[:, 0]
lead1[100] = signals[:, 1]

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0/100.hea'

In [ ]:
# Visualization of 1 patients signal and info
print(type(lead0[100]))
print(lead0[100].shape)
plt.plot(lead0[100])
print(info)

In [ ]:
# ECG signal per second
a = lead0[100][0: 3000]
plt.figure(figsize=(12, 4), dpi=90)
plt.plot(a)

### **c. All patients' ECG loading**

In [ ]:
# Loading all patients ECG SIGNALs using for loop
def get_ecg_signals(patient_ids):
    lead0 = {}
    lead1 = {}
    for id_ in patient_ids:
        signals, info = wfdb.io.rdsamp(osj(DATA_ROOT, str(id_)))
        lead0[id_] = signals[:, 0]
        lead1[id_] = signals[:, 1]
        print(f'Signal of patient {id_} extracted')
    return lead0, lead1

In [ ]:
# Loading all patient ECG INFORMATION
def get_ecg_info(patient_ids):
    _, info = wfdb.io.rdsamp(osj(DATA_ROOT, str(patient_ids)))
    resolution = 2**11  # Number of possible signal values we can have.
    info["resolution"] = 2**11
    return info

In [ ]:
lead0, lead1 = get_ecg_signals(patient_ids)

In [ ]:
# Plot any patient signal from any time frame
patient_id = "100" # can change
starting_time = 0 # can change
ending_time = 10 # can change

# Scaling
starting_signal_point = starting_time*350
ending_signal_point = ending_time*350 # As sampling frequency is 350 Hz
x = np.arange(starting_time, ending_time, 1/350)
signal = lead0[patient_id][starting_signal_point: ending_signal_point]

plt.figure(figsize=(12, 3), dpi=100)
plt.plot(x, signal)
plt.title(f'ECG signla of patient {patient_id}')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (mV)')

KeyError: '100'

In [ ]:
# ECG info of any patient
ecg_info = get_ecg_info(patient_ids[0])
ecg_info

# **Part B: Denoising, R-Peak Detection, Segmentation**

### **B1: Denoising**
Noise removing by using Discrete Wavelet Transform (DCT)

In [ ]:
# User defined fucntion for DWT and reconstruction
def denoise(data):
    # wavelet transform
    coeffs = pywt.wavedec(data=data, wavelet='db5', level=9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs

    # Threshold denoising
    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    # Inverse wavelet transform to obtain the denoised signal
    rdata = pywt.waverec(coeffs=coeffs, wavelet='db5')
    return rdata

In [ ]:
# Ploting a signal before denoising
record = wfdb.rdrecord(project_path + '100', channel_names=['MLII'])
data = record.p_signal.flatten()
plt.plot(data[0:500])

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0/100.hea'

In [ ]:
# Same signal after denoising
rdata = denoise(data=data)
plt.plot(rdata[0:500])

### **B2: R-Peak Detection**
R-peak is annotated in MIT-BIH dataset. Just need to read the file.

In [ ]:
# For exmaple, we extract '100' recording annotation
annotation = wfdb.rdann(project_path + '100', 'atr')
Rlocation = annotation.sample
print(Rlocation)
Rclass = annotation.symbol
print(Rclass)

In [ ]:
len(annotation.symbol)

In [ ]:
# R-peak ploting
x = np.arange(1, 1081)

n_peak =5
r_peak_x = []
r_peak_y = []
for i in range(0, n_peak):
  r_peak_x.append(Rlocation[i])
  r_peak_y.append(rdata[Rlocation[i]])

plt.plot(x, data[0:1080], color='red')
plt.scatter(r_peak_x, r_peak_y)

### **B3: Segmentation**
Each ECG signal is segmented by using a window **length of 300**. From R-peak location, **99** samples taken from **left** and **201** samples from **right**. Thus a complete **heartbeat** is found.

In [ ]:
# Plotting 3 heartbeats
k = np.arange(100, 103)
for i in k:
  # print(i)
  # print(Rlocation[i] - 99, Rlocation[i] + 201)
  x_train = rdata[Rlocation[i] - 99:Rlocation[i] + 201]
  plt.plot(x_train)
  print(x_train.shape)
plt.show()

### **B4: Complete Preprocessing Figures**
The complete preprocessing including denosinsing, R-peak location detection and segmentation is expected to view in a single figure.

In [ ]:
r_peak_xx = Rlocation[0], Rlocation[1], Rlocation[2], Rlocation[3]
r_peak_yy = rdata[Rlocation[0]], rdata[Rlocation[1]], rdata[Rlocation[2]], rdata[Rlocation[3]]

In [ ]:
# Plotting R-peaks and segmentation lines
fig = plt.figure(figsize=(10,3), dpi=600)
n_peak =5
r_peak_x = []
r_peak_y = []
for i in range(0, n_peak):
  r_peak_x.append(Rlocation[i])
  r_peak_y.append(rdata[Rlocation[i]])
x = np.arange(1, 1081)
plt.plot(x, rdata[0: 1080], color='red')
plt.scatter(r_peak_x, r_peak_y)

# line plotting
plt.axvline(x = Rlocation[2], color = 'k', linestyle = ':')
plt.axvline(x = Rlocation[2]-99, color = 'k', linestyle = '--')
plt.axvline(x = Rlocation[2]+201, color = 'k', linestyle = '--')

In [ ]:
# Plot together raw, denoised and segmted signal
fig = plt.figure(figsize=(10,9), dpi=600)
x = np.arange(1, 1081)

# Raw signal plotting
plt.subplot(3, 1, 1)
plt.plot(x/360, data[0:1080], color='red')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)')
plt.title('Raw ECG signal')

# Denoised signal plotting
plt.subplot(3, 1, 2)
plt.plot(x/360, rdata[0:1080], color='red')
plt.title('Denoised ECG signal')

# Segmentation visualization using two border lines
plt.subplot(3, 1, 3)
n_peak =5
r_peak_x = []
r_peak_y = []
for i in range(0, n_peak):
  r_peak_x.append(Rlocation[i])
  r_peak_y.append(rdata[Rlocation[i]])
x = np.arange(1, 1081)
plt.plot(x, rdata[0: 1080], color='red')
plt.scatter(r_peak_x, r_peak_y)
# line plotting
plt.axvline(x = Rlocation[2], color = 'k', linestyle = ':') # 3rd r-peak
plt.axvline(x = Rlocation[2]-99, color = 'k', linestyle = '--')
plt.axvline(x = Rlocation[2]+201, color = 'k', linestyle = '--')

plt.xlabel('# Sample')
plt.ylabel('Voltage (mV)')
plt.title('Segmentation using 3rd R-peak')

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)

# figure_path = '/content/gdrive/MyDrive/Heartbeat_Figures/'
# fig.savefig(figure_path+ 'Denoised and segmented ECG.png')

# **Part C: Dataset Loading**

## **C1: Loading whole data**

In [ ]:
# Read ECG signals and corresponding label
def getDataSet(number, X_data, Y_data):

    # Considering 15 types ECG heartbeats that are later grouped in 5 classes
    ecgClassSet = ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', '/', 'f', 'Q']

    # Reading Channel names
    _, info = wfdb.io.rdsamp(osj(project_path, number))
    channels = info['sig_name']
    channel1, channel2 = channels[0], channels[1]
    print(channel1, channel2)


    # Read ECG data records
    print("reading " + number+ " ECG data...")
    record = wfdb.rdrecord(project_path + number, channel_names=[channel1])
    data = record.p_signal.flatten()
    rdata = denoise(data=data)

    # Obtain the position and corresponding label of the R wave in the ECG data record
    annotation = wfdb.rdann(project_path + number, 'atr')
    Rlocation = annotation.sample
    Rclass = annotation.symbol

    # Unstable data before and after removal
    start = 2  # if it creates problem then except will do the job
    end = 3
    i = start
    j = len(annotation.symbol) - end

    # Making labels, Y_data Convert NSVFQ in order to 0123456...14
    while i < j:
        try:
            beat_type = Rclass[i]
            lable = ecgClassSet.index(beat_type)  # when beat is like '+' or other it will go on except loop
            x_train = rdata[Rlocation[i] - 99:Rlocation[i] + 201]
            X_data.append(x_train)
            Y_data.append(lable)
            i += 1
        except ValueError:
            # print(f' when i = {i}, beat type is out of our choise. For example +, [, ! or other')
            i += 1
    return X_data, Y_data

In [ ]:
# Load the dataset and preprocess it
def loadData():
    numberSet = ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109',
                 '111', '112', '113', '114', '115', '116', '117', '118', '119', '121',
                 '122', '123', '124', '200', '201', '202', '203', '205', '207', '208',
                 '209', '210', '212', '213', '214', '215', '217', '219', '220', '221',
                 '222', '223', '228', '230', '231', '232', '233', '234'] # 48 readings
    dataSet = []
    lableSet = []
    for n in numberSet:
        # getDataSet(n, dataSet, lableSet)
        dataSet, lableSet = getDataSet(n, dataSet, lableSet)

    # Turn numpy array, scramble the order
    dataSet = np.array(dataSet).reshape(-1, 300)
    lableSet = np.array(lableSet).reshape(-1, 1)
    train_ds = np.hstack((dataSet, lableSet))
    np.random.shuffle(train_ds)

    # dataset and its label set
    X = train_ds[:, :300]
    Y = train_ds[:, 300]
    return X, Y

In [ ]:
# Input X and Output Y data loading
X, Y = loadData()

In [ ]:
# Counting the number of each type of heartbeats
from collections import Counter
Y_list = list(Y)
Counter(Y_list)

## **C2: Ploting 15 Different Heartbeats**

In [ ]:
# making pandas dataframe
df_X = pd.DataFrame(X)
df_Y = pd.DataFrame(Y)

In [ ]:
# changing the name from 0 to 300
df_Y.rename(columns = {0:300}, inplace = True)
# join X and Y
df = pd.concat([df_X, df_Y], axis=1)

In [ ]:
def Plot_Random_Beat(type, num):

  ecgClassSet = ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', 'slash', 'f', 'Q']

  ecgClassName = ['Normal (N)', 'Left bundle br. bl. (L)', 'Right bundle br. bl. (R)',
                  'Atrial escape (e)', 'Nodal jun. esc. (j)', 'Atrial premature (A)',
                  'Aberrated atrial prem. (a)', 'Nodal jun. pre. (J)',
                  'Supraventricular prem. (S)', 'Premature ventr. (V)',
                  'Ventricular escape (E)', 'Fusion of ve. & no. (F)',
                  'Paced (/)', 'Fusion of pa. & no. (f)',
                  'Unclassifiable(Q)']

  # getting only a specific class ECG signal
  df_0 = df.loc[df[300]==type]  # For normanl class: 0, shape is 74920,301
  df_0 = df_0.drop(columns=[300]) # changing the shape to 74920,300

  # selecting some random row to plot
  if num<=df_0.shape[0]:
    np.random.seed(234)
    random_beat_number = np.random.randint(df_0.shape[0], size=(num))
    random_beat_number = list(random_beat_number)
  else: # Needed for Supraventricular Premature Beat (S) only, as it contains only 2 beats
    print(f"Warning: You have only {df_0.shape[0]} beat, but asked to plot {num}")
    random_beat_number = np.arange(0, df_0.shape[0])
    random_beat_number = list(random_beat_number)

  # ploting the ECG signal
  for i in random_beat_number:
    ecg_beat = df_0.iloc[i]
    plt.plot(ecg_beat)
  plt.title(str(ecgClassName[type]))

In [ ]:
# Plotting 15 different types of heartbeat
fig = plt.figure(figsize=(16,7), dpi=400)
fig.tight_layout(pad=15.0)
for i in range(15):
  plt.subplot(3,5,i+1)
  plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
  Plot_Random_Beat(type=i, num=10)
# figure_path = '/content/gdrive/MyDrive/ECG Arrhythmia trying/Heartbeat_Figures/'
# fig.savefig(figure_path+ 'all_heartbeats.png')

# **Part D: Train-Test Splitting and Class Balancing**

## **D1: Data loading**
Data is already loaded, **this step can be skipped.** However, here the whole dataset is saved in a train_ds variable.

### **a. Load whole data**

In [ ]:
# Load the dataset and preprocess it
def loadData():
    numberSet = ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109',
                 '111', '112', '113', '114', '115', '116', '117', '118', '119', '121',
                 '122', '123', '124', '200', '201', '202', '203', '205', '207', '208',
                 '209', '210', '212', '213', '214', '215', '217', '219', '220', '221',
                 '222', '223', '228', '230', '231', '232', '233', '234']  # 48 readings
    dataSet = []
    lableSet = []
    for n in numberSet:
        # getDataSet(n, dataSet, lableSet)
        dataSet, lableSet = getDataSet(n, dataSet, lableSet)

    # Turn numpy array, scramble the order
    dataSet = np.array(dataSet).reshape(-1, 300)
    lableSet = np.array(lableSet).reshape(-1, 1)
    train_ds = np.hstack((dataSet, lableSet))
    np.random.shuffle(train_ds)
    return train_ds

In [ ]:
# Load the whole dataset (109305,301). Each row indicate an ECG beat time series data upto 300
# and 301 colum is its label among 15 difference level
train_ds = loadData()

In [ ]:
Y = train_ds[:, 300]

In [ ]:
# Here 15 class of ECG data are saved
Y_list = list(Y)
Counter(Y_list)

### **b. 15 types to 5 level conversion**

In [ ]:
# 15 level to 5 level conversion
Y_5class = np.copy(Y)

for i in range(Y.shape[0]):
  # print(i)
  if 0 <= Y[i] <= 4:
    Y_5class[i] = 0
  if 5 <= Y[i] <= 8:
    Y_5class[i] = 1
  if 9 <= Y[i] <= 10:
    Y_5class[i] = 2
  if Y[i] == 11:
    Y_5class[i] = 3
  if 12 <= Y[i] <= 14:
    Y_5class[i] = 4
print('changing done')

In [ ]:
Y_5class_list = list(Y_5class)
Counter(Y_5class_list)

In [ ]:
ecg_dataset = np.copy(train_ds)

In [ ]:
# label encode the target variable # just convert numpy.float64 to numpy.int64
from sklearn.preprocessing import LabelEncoder
Y_5class = LabelEncoder().fit_transform(Y_5class)

In [ ]:
ecg_data = ecg_dataset[:, :300]
ecg_lable = Y_5class.reshape(-1, 1) # otherwise np.hstack will not work

In [ ]:
# Complete ECG dataset with 5 type of Arrhythmia
ecg_dataset_5 = np.hstack((ecg_data, ecg_lable))

### **c. Per class data status checking (Full data)**

In [ ]:
# Convert ndarray to dataframe
df_ecg = pd.DataFrame(ecg_dataset_5)
class_data = df_ecg[300].value_counts()
class_data

In [ ]:
# per class data status plotting,
plt.bar(class_data.index, class_data.values, color ='maroon')
plt.show()

In [ ]:
# shortcut for per class data status plotting,
# Order is not maintained by class. Higher to lower
df_ecg[300].value_counts().plot(kind='bar')

## **D2: Train-Test Spliting**
**Note: Class Balance should be done on Training Data Only. Not Testing Data.**

In [ ]:
# train test splitting
from sklearn.model_selection import train_test_split
ecg_data = ecg_dataset_5[:, :300]
ecg_label = ecg_dataset_5[:, 300]
x_train, x_test, y_train, y_test = train_test_split(ecg_data, ecg_label,
                                   random_state=104,
                                   test_size=0.20,
                                   shuffle=True)

In [ ]:
# reshaping for using hstack function
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
train_data = np.hstack((x_train, y_train))
test_data = np.hstack((x_test, y_test))

In [ ]:
#  converting dataframe
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [ ]:
# saving the test data (in imbalanced condition)
file_name = project_path + 'test_data.pkl'
test_data.to_pickle(file_name)

**Training dataset status checking:** balanced / imbalanced

In [ ]:
# Imblanced training data graph ploting
class_data = train_data[300].value_counts()
print(class_data)
plt.bar(class_data.index, class_data.values, color ='maroon')
plt.show()

## **D3: Class balancing by undersampling and SMOTE**
**SMOTE** stands for '**Synthetic Minority Oversampling Technique**'.
Plan for train data
1. Class 1: Randomly selected 50000 data
2. Class 1, 2, 3, 4: Use SMOTE to oversample upto 50000 data

In [ ]:
# extracting class 0 and 4 others class
train_data_0 = train_data.loc[(train_data[300] == 0)]
train_data_1234 = train_data.loc[(train_data[300] != 0)]

In [ ]:
# 1. Class 1: Randomly selected 50000 data
from sklearn.utils import resample
train_data_0_resampled=train_data_0.sample(n=50000,random_state=42)

# convert dataframe to numpy array
train_data_0_resampled = train_data_0_resampled.to_numpy()

In [ ]:
# 2. Class 1, 2, 3, 4: Use SMOTE to oversample upto 50000 data

# converting from df to np ndarray
train_data_1234_arr = train_data_1234.to_numpy()
X_4cl, y_4cl = train_data_1234_arr[:, :-1], train_data_1234_arr[:, -1]

from imblearn.over_sampling import SMOTE
# transform the dataset
strategy = {1:50000, 2:50000, 3:50000, 4:50000}
oversample = SMOTE(sampling_strategy=strategy)
X, y = oversample.fit_resample(X_4cl, y_4cl)

y = y.reshape(-1, 1)
train_data_1234_resampled = np.hstack((X, y))

In [ ]:
# Join the class 0 and 1234
train_data_resampled = np.vstack((train_data_0_resampled, train_data_1234_resampled))

# shuffle the data, needed for proper training
np.take(train_data_resampled,np.random.permutation(train_data_resampled.shape[0]),axis=0,out=train_data_resampled)

In [ ]:
# blanced training data graph ploting
train_data_r = pd.DataFrame(train_data_resampled)
class_data = train_data_r[300].value_counts()
print(class_data)
plt.bar(class_data.index, class_data.values, color ='maroon')
plt.show()

# save balanced training data
file_name = project_path + 'train_data_SMOTE.pkl'
train_data_r.to_pickle(file_name)

In [ ]:
data_bal = np.array(class_data)
data_bal2 = data_bal.reshape(1, 5)

In [ ]:
# a single plot which gives proper illustration before and after class balancing
import seaborn as sns
sns.set()
sns.color_palette("hls", 8)

fig = plt.figure(figsize=(7,4), dpi=600)
plt.subplot(121)
sns.barplot(x = ['N', 'S', 'V', 'F', 'Q'], y = [72420, 2212, 5774, 637, 6401])
plt.ylim(0, 75000)
plt.title('Training Data, Imbalanced')

plt.subplot(122)
sns.barplot(x = ['N', 'S', 'V', 'F', 'Q'], y = class_data.values)
plt.ylim(0, 75000)
plt.title('Balanced by SMOTE')

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.5)

# figure_path = '/content/gdrive/MyDrive/ECG Arrhythmia trying/Heartbeat_Figures/'
# fig.savefig(figure_path+ 'Class balancing.png')

In [ ]:
# --- Build final arrays for the model zoo ---

# 1) TRAIN from your balanced SMOTE set
# If you already have train_data_resampled in memory, you can use it directly.
# Otherwise, load the pickle saved earlier:
try:
    train_df = pd.read_pickle(project_path + 'train_data_SMOTE.pkl')
    train_np = train_df.to_numpy()
except Exception:
    # Fallback if not saved as pickle in your run
    train_np = train_data_resampled  # from previous cells

X_train_full = train_np[:, :300]
y_train_full = train_np[:, 300].astype('int64')

# Add channel dimension for 1D convs: (N, 300, 1)
X_train_full = X_train_full[..., None]

# 2) TEST from the imbalanced holdout you saved earlier
test_df = pd.read_pickle(project_path + 'test_data.pkl')
test_np = test_df.to_numpy()

X_test = test_np[:, :300][..., None]
y_test = test_np[:, 300].astype('int64')

# 3) Split TRAIN into train/val (stratified)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.15, random_state=42, stratify=y_train_full
)

print(X_train.shape, X_val.shape, X_test.shape)
print(np.bincount(y_train), np.bincount(y_val), np.bincount(y_test))


# **Part E: Model Building and Training**
A **CNN-LSTM and attention** based hybrid model is formulated.

## **E2: CNN-LSTM and attention model architecture**

In [ ]:
import os, time, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import pandas as pd
np.random.seed(42); tf.random.set_seed(42)

input_shape = X_train.shape[1:]         # (seq_len, 1)
num_classes = len(np.unique(y_train))

def common_tail(x, num_classes):
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.3)(x)
    return layers.Dense(num_classes, activation="softmax")(x)

def compile_model(inputs, outputs, lr=1e-3):
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# --- Model zoo ---

def build_cnn1d(filters=[32,64,128], kernel=7):
    inp = keras.Input(shape=input_shape)
    x = inp
    for f in filters:
        x = layers.Conv1D(f, kernel, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.MaxPooling1D(2)(x)
        x = layers.Dropout(0.2)(x)
    out = common_tail(x, num_classes)
    return compile_model(inp, out)

def build_lstm(units=128):
    inp = keras.Input(shape=input_shape)
    x = layers.LSTM(units, return_sequences=False)(inp)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return compile_model(inp, out)

def build_bilstm(units=96):
    inp = keras.Input(shape=input_shape)
    x = layers.Bidirectional(layers.LSTM(units))(inp)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return compile_model(inp, out)

def build_gru(units=128):
    inp = keras.Input(shape=input_shape)
    x = layers.GRU(units)(inp)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return compile_model(inp, out)

def build_bigru(units=96):
    inp = keras.Input(shape=input_shape)
    x = layers.Bidirectional(layers.GRU(units))(inp)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return compile_model(inp, out)

# Simple TCN block (dilated causal conv residual stack)
def tcn_block(x, filters, kernel=3, dilation=1, dropout=0.2):
    y = layers.Conv1D(filters, kernel, padding="causal", dilation_rate=dilation)(x)
    y = layers.BatchNormalization()(y); y = layers.ReLU()(y)
    y = layers.Dropout(dropout)(y)
    y = layers.Conv1D(filters, kernel, padding="causal", dilation_rate=dilation)(y)
    y = layers.BatchNormalization()(y)
    if x.shape[-1] != filters:
        x = layers.Conv1D(filters, 1, padding="same")(x)
    y = layers.Add()([x, y]); y = layers.ReLU()(y)
    return y

def build_tcn(filters=64, stacks=3, kernel=3):
    inp = keras.Input(shape=input_shape)
    x = inp
    for s in range(stacks):
        for d in [1, 2, 4, 8]:  # dilations
            x = tcn_block(x, filters, kernel, dilation=d)
    out = common_tail(x, num_classes)
    return compile_model(inp, out)

# ResNet1D-18 (1D adaptation)
def res_block_1d(x, f, stride=1):
    y = layers.Conv1D(f, 3, padding="same", strides=stride)(x)
    y = layers.BatchNormalization()(y); y = layers.ReLU()(y)
    y = layers.Conv1D(f, 3, padding="same")(y)
    y = layers.BatchNormalization()(y)
    if stride != 1 or x.shape[-1] != f:
        x = layers.Conv1D(f, 1, strides=stride, padding="same")(x)
        x = layers.BatchNormalization()(x)
    y = layers.Add()([x, y]); y = layers.ReLU()(y)
    return y

def build_resnet1d18():
    inp = keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 7, strides=2, padding="same")(inp)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.MaxPooling1D(3, strides=2, padding="same")(x)
    # layers: [2,2,2,2]
    for f, n, s in [(64,2,1),(128,2,2),(256,2,2),(512,2,2)]:
        for i in range(n):
            x = res_block_1d(x, f, stride=(s if i==0 else 1))
    out = common_tail(x, num_classes)
    return compile_model(inp, out)

# VGG1D-16 (stacked convs + pooling)
def vgg_block_1d(x, filters, convs=2):
    for _ in range(convs):
        x = layers.Conv1D(filters, 3, padding="same")(x)
        x = layers.ReLU()(x)
    return layers.MaxPooling1D(2)(x)

def build_vgg1d16():
    inp = keras.Input(shape=input_shape)
    x = inp
    x = vgg_block_1d(x, 64, 2)
    x = vgg_block_1d(x, 128, 2)
    x = vgg_block_1d(x, 256, 3)
    x = vgg_block_1d(x, 512, 3)
    x = vgg_block_1d(x, 512, 3)
    out = common_tail(x, num_classes)
    return compile_model(inp, out)

# InceptionTime (1D GoogLeNet-ish)
def inception_time_module(x, nb_filters=32):
    b1 = layers.Conv1D(nb_filters, 9, padding='same')(x)
    b2 = layers.Conv1D(nb_filters, 19, padding='same')(x)
    b3 = layers.Conv1D(nb_filters, 39, padding='same')(x)
    b4 = layers.MaxPooling1D(3, padding='same', strides=1)(x)
    b4 = layers.Conv1D(nb_filters, 1, padding='same')(b4)
    x = layers.Concatenate()([b1, b2, b3, b4])
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    return x

def build_inceptiontime(depth=6, nb_filters=32):
    inp = keras.Input(shape=input_shape)
    x = inp
    for _ in range(depth):
        x = inception_time_module(x, nb_filters)
    out = common_tail(x, num_classes)
    return compile_model(inp, out)

MODEL_ZOO = {
    "CNN1D": build_cnn1d,
    "LSTM": build_lstm,
    "BiLSTM": build_bilstm,
    "GRU": build_gru,
    "BiGRU": build_bigru,
    "TCN": build_tcn,
    "ResNet1D-18": build_resnet1d18,
    "VGG1D-16": build_vgg1d16,
    "InceptionTime": build_inceptiontime,  # stand-in for GoogLeNet style
}

# --- Callbacks & training loop ---

def callbacks(name):
    return [
        keras.callbacks.ModelCheckpoint(
            f"best_{name}.keras", save_best_only=True, monitor="val_accuracy", mode="max"
        ),
        keras.callbacks.ReduceLROnPlateau(
            patience=5, factor=0.5, min_lr=1e-6, monitor="val_loss"
        ),
        keras.callbacks.EarlyStopping(
            patience=10, restore_best_weights=True, monitor="val_accuracy", mode="max"
        ),
    ]





results = []

for name, builder in MODEL_ZOO.items():
    print(f"\n=== Training {name} ===")
    model = builder()

    start = time.time()
    hist = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20, batch_size=128,
        callbacks=callbacks(name),
        verbose=2,
        class_weight=None  # set your class_weight dict if needed
    )
    train_time = time.time() - start

    # Evaluate
    y_prob = model.predict(X_test, verbose=0)
    y_hat = np.argmax(y_prob, axis=1)
    acc = (y_hat == y_test).mean()
    f1_macro = f1_score(y_test, y_hat, average='macro')
    report = classification_report(y_test, y_hat, output_dict=True, zero_division=0)
    cm = confusion_matrix(y_test, y_hat)

    # Save confusion matrix and report
    pd.DataFrame(cm).to_csv(f"cm_{name}.csv", index=False)
    pd.DataFrame(report).T.to_csv(f"report_{name}.csv")

    # Params
    params = model.count_params()

    results.append({
        "model": name,
        "params": params,
        "test_acc": acc,
        "test_f1_macro": f1_macro,
        "train_time_sec": int(train_time),
        "best_ckpt": f"best_{name}.keras"
    })

df = pd.DataFrame(results).sort_values("test_f1_macro", ascending=False)
df.to_csv("results.csv", index=False)
df


In [ ]:
# ================================================
# Part F2: CNN–GRU fusion sweep (sequential | parallel | interleaved | multiscale)
# ================================================
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize

# ---------- Small helpers ----------
def _ensure_3d(X):
    """Make sure X is (N, seq_len, 1)."""
    return X if X.ndim == 3 else X[..., None]

def _callbacks(tag):
    return [
        keras.callbacks.ModelCheckpoint(
            f"best_cnn_gru_{tag}.keras", save_best_only=True,
            monitor="val_accuracy", mode="max"
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", patience=4, factor=0.5, min_lr=1e-5
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_accuracy", mode="max", patience=8, restore_best_weights=True
        ),
    ]

def _metrics(y_true, y_prob):
    y_pred = np.argmax(y_prob, axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro", zero_division=0)
    # AUROC (OvR) if >2 classes
    C = y_prob.shape[1]
    try:
        Yb = label_binarize(y_true, classes=np.arange(C))
        auroc = roc_auc_score(Yb, y_prob, average="macro", multi_class="ovr") if C > 2 \
                else roc_auc_score(y_true, y_prob[:,1])
    except Exception:
        auroc = float("nan")
    return acc, f1m, auroc

# ---------- Basic blocks ----------
def conv_block(x, filters, k, pool=True, drop=0.0):
    x = layers.Conv1D(filters, k, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    if pool:
        x = layers.MaxPooling1D(2)(x)
    if drop > 0:
        x = layers.Dropout(drop)(x)
    return x

def multi_scale_cnn(x, nf=32):
    b1 = layers.Conv1D(nf, 3,  padding="same")(x)
    b2 = layers.Conv1D(nf, 7,  padding="same")(x)
    b3 = layers.Conv1D(nf, 15, padding="same")(x)
    x  = layers.Concatenate()([b1, b2, b3])
    x  = layers.BatchNormalization()(x)
    x  = layers.ReLU()(x)
    return x

# ---------- Fusion builders ----------
def build_cnn_gru_fusion(fusion: str,
                         seq_input_shape,
                         n_classes: int,
                         gru_units: int = 64) -> keras.Model:
    """
    fusion in {"sequential","parallel","interleaved","multiscale"}
    """
    inp = keras.Input(shape=seq_input_shape)

    if fusion == "sequential":
        # CNN -> (Bi)GRU -> GAP -> Dense
        x = conv_block(inp, 32, 7, pool=True,  drop=0.1)
        x = conv_block(x,   64, 5, pool=True,  drop=0.1)
        x = layers.SpatialDropout1D(0.1)(x)
        x = layers.Bidirectional(layers.GRU(gru_units, return_sequences=True, dropout=0.2))(x)
        x = layers.LayerNormalization()(x)
        x = layers.GlobalAveragePooling1D()(x)

    elif fusion == "parallel":
        # Branch A: CNN -> GAP
        ca = conv_block(inp, 32, 7, pool=True,  drop=0.1)
        ca = conv_block(ca,  64, 5, pool=True,  drop=0.1)
        ca = layers.GlobalAveragePooling1D()(ca)
        # Branch B: (Bi)GRU -> last hidden (or GAP of sequences)
        cb = layers.Bidirectional(layers.GRU(gru_units, return_sequences=False, dropout=0.2))(inp)
        # Fuse
        x = layers.Concatenate()([ca, cb])

    elif fusion == "interleaved":
        # CNN -> (Bi)GRU -> CNN -> GAP
        x = conv_block(inp, 32, 7, pool=True,  drop=0.1)
        x = layers.Bidirectional(layers.GRU(gru_units, return_sequences=True, dropout=0.2))(x)
        x = layers.LayerNormalization()(x)
        x = layers.Conv1D(64, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.GlobalAveragePooling1D()(x)

    elif fusion == "multiscale":
        # Multi-kernel CNN -> (Bi)GRU -> GAP
        x = multi_scale_cnn(inp, nf=32)
        x = layers.MaxPooling1D(2)(x)
        x = layers.SpatialDropout1D(0.1)(x)
        x = layers.Bidirectional(layers.GRU(gru_units, return_sequences=True, dropout=0.2))(x)
        x = layers.GlobalAveragePooling1D()(x)

    else:
        raise ValueError(f"Unknown fusion mode: {fusion}")

    x = layers.Dropout(0.3)(x)
    out = layers.Dense(n_classes, activation="softmax")(x)

    model = keras.Model(inp, out, name=f"CNN_GRU_{fusion}")
    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

# ---------- Select which split you want to use ----------
# If you created both intra and inter earlier, pick one set here:
# e.g., INTRA:
# X_train, y_train, X_val, y_val, X_test, y_test = X_train_intra, y_train_intra, X_val_intra, y_val_intra, X_test_intra, y_test_intra
# or INTER:
# X_train, y_train, X_val, y_val, X_test, y_test = X_train_inter, y_train_inter, X_val_inter, y_val_inter, X_test_inter, y_test_inter

# Ensure shapes are (N, seq_len, 1)
Xt = _ensure_3d(X_train); Xv = _ensure_3d(X_val); Xte = _ensure_3d(X_test)

# ---------- Run all fusion modes ----------
fusion_modes = ["sequential", "parallel", "interleaved", "multiscale"]
results = []

for mode in fusion_modes:
    print(f"\n=== Training CNN–GRU ({mode}) ===")
    model = build_cnn_gru_fusion(
        fusion=mode,
        seq_input_shape=Xt.shape[1:],   # (seq_len, 1)
        n_classes=int(len(np.unique(y_train))),
        gru_units=64
    )

    hist = model.fit(
        Xt, y_train,
        validation_data=(Xv, y_val),
        epochs=15,
        batch_size=128,
        callbacks=_callbacks(mode),
        verbose=2
    )

    # Evaluate
    y_prob = model.predict(Xte, verbose=0)
    acc, f1m, auroc = _metrics(y_test, y_prob)

    best_e = int(np.argmax(hist.history["val_accuracy"]))
    best_va = float(hist.history["val_accuracy"][best_e])

    results.append({
        "fusion_mode": mode,
        "params": int(model.count_params()),
        "best_val_acc": best_va,
        "test_acc": float(acc),
        "f1_macro": float(f1m),
        "auroc_macro_ovr": float(auroc) if not np.isnan(auroc) else np.nan,
        "best_epoch": best_e + 1
    })

# ---------- Comparison table ----------
df_cnn_gru = pd.DataFrame(results).sort_values(["f1_macro","test_acc"], ascending=False)
display(df_cnn_gru)
df_cnn_gru.to_csv("cnn_gru_fusion_results.csv", index=False)
print("\nSaved: cnn_gru_fusion_results.csv")


# **Part F: Results**


## **F1: Classification Accuracy and Confusion Matrix**
The overall classification accuracy and confusion matrix generated by the follwoing code.

In [ ]:
import pandas as pd

df = pd.read_csv("results.csv").sort_values("test_f1_macro", ascending=False)
print("Model leaderboard (sorted by Macro-F1):")
df


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

best = df.iloc[0]["model"]
print("Best model:", best)

cm = pd.read_csv(f"cm_{best}.csv").to_numpy()

# Try to use your test labels if they exist; else index labels
try:
    label_ids = np.unique(y_test)
    labels = [str(i) for i in label_ids]
except NameError:
    labels = [str(i) for i in range(cm.shape[0])]

plt.figure(figsize=(6,6))
plt.imshow(cm, interpolation="nearest")
plt.title(f"Confusion Matrix — {best}")
plt.xticks(np.arange(len(labels)), labels)
plt.yticks(np.arange(len(labels)), labels)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

best = df.iloc[0]["model"]
cm = pd.read_csv(f"cm_{best}.csv").to_numpy()

cm_row = cm / cm.sum(axis=1, keepdims=True)  # normalize by true counts (recall)
labels = ['0','1','2','3','4']  # or ['N','S','V','F','Q'] if that’s your mapping

plt.figure(figsize=(6,6))
plt.imshow(cm_row, vmin=0, vmax=1)
plt.title(f'Confusion Matrix (row-normalized) — {best}')
plt.xticks(np.arange(len(labels)), labels)
plt.yticks(np.arange(len(labels)), labels)
for i in range(cm_row.shape[0]):
    for j in range(cm_row.shape[1]):
        plt.text(j, i, f"{cm_row[i, j]*100:.1f}%", ha="center", va="center")
plt.xlabel('Predicted'); plt.ylabel('True'); plt.tight_layout(); plt.show()
